# Calculate Simplified Ikeda simulation

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 7

from rolldecayestimators.simplified_ikeda import calculate_roll_damping
from rolldecayestimators import equations
import sympy as sp
from rolldecayestimators import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify
from mdldb.tables import Run
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.ikeda_estimator import IkedaEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from sklearn.pipeline import Pipeline
from rolldecayestimators.sensitivity import variate_ship, plot_variation, calculate, calculate_variation, _plot_result

import data
import copy
from rolldecay import database
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

In [ ]:
db = database.get_db()

In [ ]:
required = ['lpp','beam','TA','TF','BKL','BKB','omega0','kg','Volume','A0']

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_linear')

df_rolldecay.fillna(0, inplace=True)
mask = ((df_rolldecay['ship_speed']==0) &  ## Zero speed!
        (df_rolldecay[required] > 0).all(axis=1)
       )

df = df_rolldecay.loc[mask].copy()
df['phi_max'] = np.rad2deg(df['phi_start'].abs())
df['DRAFT'] = (df.TA + df.TF)/2
df['CB'] = df.Volume/(df.lpp*df.beam*df.DRAFT)

In [ ]:
row = df.iloc[0]

In [ ]:
db_run = db.session.query(Run).get(int(row.id))

In [ ]:
df = database.load_run(db_run=db_run)

In [ ]:
df.plot(y='phi')

In [ ]:
result = calculate(row)
result

In [ ]:
Math(vlatex(equations.B44_equation))

In [ ]:
sp.Eq(symbols.B_44, sp.solve(equations.B44_equation, symbols.B_44)[0])

In [ ]:
B44_lambda = lambdify(sp.solve(equations.B44_equation, symbols.B_44)[0])

In [ ]:
g=9.81
rho=1000
B_44 = B44_lambda(B_44_hat=result.B44HAT, Disp=row.Volume, beam=row.beam, g=g, rho=rho)

Equivalent linearized damping according to [Himeno]:

In [ ]:
equations.B_e_equation

In [ ]:
equations.zeta_equation

In [ ]:
equations.omega0_equation_linear

In [ ]:
eqs = [equations.zeta_equation,
      equations.omega0_equation_linear]

In [ ]:
zeta_lambda = lambdify(sp.solve(eqs,symbols.A_44,symbols.zeta)[0][1])

In [ ]:
m = row.Volume*rho
zeta=zeta_lambda(B_1=B_44, GM=row.gm, g=g, m=m,  omega0=row.omega0)
zeta

In [ ]:
row.zeta

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
scaler = ScaleFactorTransformer(scale_factor=None)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25))
offset_transformer = OffsetTransformer()

In [ ]:
scaler.scale_factor=row.scale_factor

In [ ]:
direct_linear_estimator = DirectLinearEstimator()

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),]
    #('linear_estimator', direct_linear_estimator)]
       
preprocessor = Pipeline(steps) # define the pipeline object.

In [ ]:
preprocessor.fit(df)
X=preprocessor.transform(df)

In [ ]:
X.plot(y='phi')

In [ ]:
estimator = DirectLinearEstimator()
t=X.index
phi0=X.iloc[0]['phi']
phi1d0=X.iloc[0]['phi1d']

df_sim_ikeda = estimator.simulate(t=t, phi0=phi0, phi1d0=phi1d0, omega0=row.omega0, zeta=zeta)
df_sim = estimator.simulate(t=t, phi0=phi0, phi1d0=phi1d0, omega0=row.omega0, zeta=row.zeta)


In [ ]:
df_result = X.copy()
df_result = pd.merge(left=df_result, right=df_sim_ikeda, how='left', left_index=True, right_index=True, 
                     suffixes =('','_ikeda'))

df_result = pd.merge(left=df_result, right=df_sim, how='left', left_index=True, right_index=True, 
                     suffixes =('','_model'))

fig,ax=plt.subplots()
df_result.plot(y='phi',label='model test', style=':', ax=ax)
df_result.plot(y='phi_model',label='model', style='-',ax=ax)
df_result.plot(y='phi_ikeda',label='ikeda', style='--',ax=ax)

## Model scale...

In [ ]:
df = database.load_run(db_run=db_run)

direct_linear_estimator = DirectLinearEstimator()
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25), phi1d_start_tolerance=10)
steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),]
    #('linear_estimator', direct_linear_estimator)]
       
preprocessor = Pipeline(steps) # define the pipeline object.

In [ ]:
preprocessor.fit(df)
X_model=preprocessor.transform(df)
X_model.plot(y='phi')

In [ ]:
row2 = row.copy()
scale=db_run.model.scale_factor

row2['lpp']/=scale
row2['beam']/=scale
row2['TA']/=scale
row2['TF']/=scale

row2['BKL']/=scale
row2['BKB']/=scale
row2['omega0']/=1/np.sqrt(scale)
#row2['OG']/=scale
row2['kg']/=scale
row2['gm']/=scale
row2['Volume']/=(scale**3)

In [ ]:
ikeda_estimator_ship = IkedaEstimator(lpp=row.lpp, 
                                 TA=row.TA, 
                                 TF = row.TF,
                                 beam = row.beam,
                                 BKL = row.BKL,
                                 BKB = row.BKB,
                                 A0 = row.A0,
                                 kg = row.kg,
                                 Volume = row.Volume,
                                 gm = row.gm)

ikeda_estimator_model = IkedaEstimator(lpp=row2.lpp, 
                                 TA=row2.TA, 
                                 TF = row2.TF,
                                 beam = row2.beam,
                                 BKL = row2.BKL,
                                 BKB = row2.BKB,
                                 A0 = row2.A0,
                                 kg = row2.kg,
                                 Volume = row2.Volume,
                                 gm = row2.gm)

In [ ]:
ikeda_estimator_ship.fit(X=X)
df_ikeda_ship = ikeda_estimator_ship.predict(X)
df_ikeda_ship.index/=np.sqrt(row.scale_factor)  # Scale to model scale time

ikeda_estimator_model.fit(X=X_model)
df_ikeda_model = ikeda_estimator_model.predict(X_model)

fig,ax=plt.subplots()
df_ikeda_ship.plot(y='phi', ax=ax, label='ship')
df_ikeda_model.plot(y='phi', ax=ax, label='model')



...the friction in model scale seem to have a small impact

In [ ]:
ikeda_estimator_ship.fit(X=X)
df_ikeda_ship = ikeda_estimator_ship.predict(X)
df_ikeda_ship.index/=np.sqrt(row.scale_factor)  # Scale to model scale time

## Looking at the quadratic part of Ikeda

In [ ]:
df = database.load_run(db_run=db_run)
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
scaler = ScaleFactorTransformer(scale_factor=row.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(2), phi_min=np.deg2rad(0.25))
offset_transformer = OffsetTransformer()

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),]
    #('linear_estimator', direct_linear_estimator)]
       
preprocessor = Pipeline(steps) # define the pipeline object.

preprocessor.fit(df)
X2=preprocessor.transform(df)

In [ ]:
ikeda_estimator_ship.fit(X2)
df_sim_ikeda = ikeda_estimator_ship.predict(X2)
direct_estimator = DirectLinearEstimator()
direct_estimator.fit(X2)
df_sim_model = direct_estimator.predict(X2)

df_result = X2.copy()
df_result = pd.merge(left=df_result, right=df_sim_ikeda, how='left', left_index=True, right_index=True, 
                     suffixes =('','_ikeda'))

df_result = pd.merge(left=df_result, right=df_sim_model, how='left', left_index=True, right_index=True, 
                     suffixes =('','_model'))

fig,ax=plt.subplots()
df_result.plot(y='phi',label='model test', style=':', ax=ax)
df_result.plot(y='phi_model',label='model', style='-',ax=ax)
df_result.plot(y='phi_ikeda',label='ikeda', style='--',ax=ax)

In [ ]:
N=40
changes = np.linspace(1,0.0,N)
df_variation = variate_ship(ship=row, key='phi_max', changes=changes)
result = calculate_variation(df=df_variation)
df_variation['g']=9.81
df_variation['rho']=1000
result = pd.concat((result,df_variation), axis=1)

In [ ]:
plot_variation(ship=row, key='phi_max', changes=changes)

In [ ]:
result['B_44'] = B44_lambda(B_44_hat=result.B44HAT, Disp=row.Volume, beam=row.beam, g=g, rho=rho)
result['zeta'] = zeta_lambda(B_1=result['B_44'], GM=row.gm, g=g, m=m,  omega0=row.omega0)
result.dropna(inplace=True)

In [ ]:
fig,ax=plt.subplots()
result.plot(y='zeta', ax=ax)
ylim=ax.get_ylim()
ax.set_ylim(0,ylim[1])

In [ ]:
B_e_lambda = lambdify(sp.solve(equations.B_e_equation_cubic,symbols.B_e)[0])

In [ ]:
def fit(df,B_1,B_2,B_3):
    omega0=df['omega0']
    phi_a=df['phi_max']
    return B_e_lambda(B_1, B_2, B_3, omega0, phi_a)

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
coeffs,_=curve_fit(f=fit,xdata=result,ydata=result['B_44'])

In [ ]:
B_1=coeffs[0]
B_2=coeffs[1]
B_3=coeffs[2]
result['B_44_fit'] = fit(result,*coeffs)

In [ ]:
fig,ax=plt.subplots()
result.plot(y='B_44', ax=ax)
result.plot(y='B_44_fit', ax=ax, style='--')


In [ ]:
B_e_lambda = lambdify(sp.solve(equations.B_e_equation,symbols.B_e)[0])

In [ ]:
def fit2(df,B_1,B_2):
    omega0=df['omega0']
    phi_a=np.deg2rad(df['phi_max'])
    return B_e_lambda(B_1, B_2, omega0, phi_a)

In [ ]:
coeffs,_=curve_fit(f=fit2,xdata=result,ydata=result['B_44'])
B_1=coeffs[0]
B_2=coeffs[1]
result['B_44_fit'] = fit2(result,*coeffs)

In [ ]:
fig,ax=plt.subplots()
result.plot(y='B_44', ax=ax)
result.plot(y='B_44_fit', ax=ax, style='--')

In [ ]:
equations.zeta_B1_equation

In [ ]:
equations.d_B2_equation

In [ ]:
zeta_B1_lambda = lambdify(sp.solve(equations.zeta_B1_equation, symbols.zeta)[0])
d_B2_lambda = lambdify(sp.solve(equations.d_B2_equation, symbols.d)[0])

In [ ]:
m=result.Volume*result.rho
result['zeta'] = zeta_B1_lambda(B_1=B_1,GM=result.gm,g=result.g, m=m, omega0=result.omega0)
result['d'] = d_B2_lambda(B_2=B_2,GM=result.gm,g=result.g, m=m, omega0=result.omega0)

In [ ]:
result.plot(y=['zeta','d'])

In [ ]:
direct_estimator = DirectEstimator()
t = np.array(X.index,)
phi0=X.iloc[0]['phi']
phi1d0=X.iloc[0]['phi1d']
omega0=row.omega0
zeta=result.iloc[0]['zeta']
d=result.iloc[0]['d']
df_ikeda_quadratic_sim=direct_estimator.simulate(t=t, phi0=phi0, phi1d0=phi1d0, omega0=omega0, 
                                                 zeta=zeta, d=d)

In [ ]:
df_result = X.copy()
df_result = pd.merge(left=df_result, right=df_ikeda_quadratic_sim, how='left', left_index=True, right_index=True, 
                     suffixes =('','_ikeda_quadratic'))

fig,ax=plt.subplots()
df_result.plot(y='phi',label='model test', style='r-', ax=ax)
df_result.plot(y='phi_ikeda_quadratic',label='ikeda_quadratic', style='--',ax=ax)